# Scraping 101

# Kittens

In [1]:
import requests 

try:
    request = requests.get("http://placekitten.com/")
    kittens = request.text
    print (kittens[559:1000])
except requests.exceptions.RequestException as e:
    print ("No kittenz. Got an error code:", e)

                      __     __,
                      \,`~"~` /
      .-=-.           /    . .\
     / .-. \          {  =    Y}=
    (_/   \ \          \      / 
           \ \        _/`'`'`b
            \ `.__.-'`        \-._
             |            '.__ `'-;_
             |            _.' `'-.__)
              \    ;_..-`'/     //  \
              |   /  /   |     //    |
              \  \ \__)   \   //    /
               \__)  


# Budapest Tickets
Мы попробуем аггрегировать информацию с разных страниц одного сайта. Здесь пример немного игрушечный, но у сайта общественного транспорта будапешта очень удобная структура, и он хорошо подходит для разогрева.

Самый простой способ собрать информацию в интернете - изъять ее из html кода страницы. Давайте попробуем провернуть эту историю с http://www.bkk.hu/en/tickets-and-passes/prices/. 

In [ ]:
import requests
web = requests.get("http://www.bkk.hu/en/tickets-and-passes/prices/")
text = web.text

Весь html код страницы содержится в строковой переменной. По сути этот тот же самый код, который вы видите в page source, если вы кликните правой кнопкой мыши в вашем браузере. О-о-о-очень длинная строка.


In [55]:
text

'<!DOCTYPE html>\n<html lang="en-US" prefix="og: http://ogp.me/ns#">\n<head>\n<meta charset="UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<title>Types and prices - Budapesti K\xc3\xb6zleked\xc3\xa9si K\xc3\xb6zpontBudapesti K\xc3\xb6zleked\xc3\xa9si K\xc3\xb6zpont</title>\n\n<link rel="stylesheet" href="https://bkk.hu/wp-content/plugins/sitepress-multilingual-cms/res/css/language-selector.css?v=200.0.4" type="text/css" media="all" />\n<link rel="profile" href="//gmpg.org/xfn/11" />\n<link rel="stylesheet" type="text/css" media="all" href="https://bkk.hu/wp-content/themes/bkk/style.css.php?v=28065928" />\n<link rel="pingback" href="https://bkk.hu/xmlrpc.php" />\n<link id="page_favicon" href="https://bkk.hu/favicon.ico" rel="icon" type="image/x-icon" />\n<script src="https://bkk.hu/wp-content/themes/bkk/js/slide.js?v=29201333" type="text/javascript"></script>\n\n<!-- This site is optimized with the Yoast SEO plugin v5.3.3 - https://yoast.com/wordpress/plugins/seo/ 

In [ ]:
text = text.encode('utf8')

Чтобы извлечь нужную информацию, нужно ориентироваться в html коде и как устроена страница в целом. Перед тем как начать, нужно посмотреть source code страницы, найти там через поиск интересующую нас информацию (в данном случае можем ориентироваться на цены на билеты). И посмотреть, есть ли какой-то набор тегов, с помощью которых мы можем отсечь нужные части строки.

In [ ]:
splitted=text.split('<div><span>') # Как результат получаем список строк, которые были разбиты по строке '<div><span>'

In [57]:
len(splitted) # 171 таких строк

171

Теперь смотрим, что внутри.

In [58]:
splitted[1]

'<a href="http://www.bkk.hu/en/prices/single-ticket/">Single ticket</a></span></div>\n</td>\n<td width="30%" align="left">\n'

Этот элемент содержит название билета.

Идем дальше.

In [59]:
splitted[2]

'HUF 350</span></div>\n</td>\n</tr>\n<tr>\n<td width="70%" align="left">\n'

Цена билета здесь!

Если нам повезло, то цена эта последовательность повторяется.

In [60]:
splitted[3]

'<a href="http://www.bkk.hu/en/prices/single-ticket-bought-on-the-spot/">Single ticket bought on the spot</a></span></div>\n</td>\n<td width="30%" align="left">\n'

In [61]:
splitted[4]

'HUF 450</span></div>\n</td>\n</tr>\n<tr>\n<td width="70%" align="left">\n'

Похоже на правду. Поэтому мы можем попрбовать взять все наши строки и вытащить из них нужную информацию. Все нечетные элементы будут содержать название билета, и четные - цену. Так же нам нужно почистить эти строки.


In [64]:
temp_substring=splitted[1].split('</span></div>')[0] ## Так мы разбиваем опять по новому разделителю и берем только первый элемент из списка.
temp_substring

'<a href="http://www.bkk.hu/en/prices/single-ticket/">Single ticket</a>'

In [65]:
temp_substring[temp_substring.find('>')+len('>'):temp_substring.rfind('<')] ## Так мы находим часть строки между символами > <

'Single ticket'

Отлично! Должно сработать со всеми строками, содержащими названиями. Теперь давайте посмотрим, что там с ценой.

In [66]:
temp_substring=splitted[2].split('</span></div>')
temp_substring=temp_substring[0]
temp_substring

'HUF 350'

Теперь давайте создадим два списка - один со всеми названиями и второй с ценами.

In [70]:
type_ticket=[]
for element in splitted[1::2]:
    print(element)
    temp=element.split('</span></div>')[0]
    ticket=temp[temp.find('>')+len('>'):temp.rfind('<')]
    print(ticket)
    type_ticket.append(ticket)

<a href="http://www.bkk.hu/en/prices/single-ticket/">Single ticket</a></span></div>
</td>
<td width="30%" align="left">

Single ticket
<a href="http://www.bkk.hu/en/prices/single-ticket-bought-on-the-spot/">Single ticket bought on the spot</a></span></div>
</td>
<td width="30%" align="left">

Single ticket bought on the spot
<a href="http://www.bkk.hu/en/block-of-10-tickets/">Block of 10 tickets</a></span></div>
</td>
<td width="30%" align="left">

Block of 10 tickets
<a href="http://bkk.hu/en/airport-shuttle-bus-single-ticket/">Airport shuttle bus single ticket </a>(100E)</span></div>
</td>
<td width="30%" align="left">

Airport shuttle bus single ticket 
<a href="http://www.bkk.hu/en/prices/transfer-ticket-2/">Transfer ticket</a></span></div>
</td>
<td width="30%" align="left">

Transfer ticket
<a href="http://www.bkk.hu/en/prices/short-section-metro-ticket/">Short section metro ticket for up to 3 stops</a></span></div>
</td>
<td width="30%" align="left">

Short section metro ticket 

In [ ]:
price=[]
for element in splitted[2::2]:
    temp=element.split('</span></div>')[0]
    price.append(temp)

А теперь с помощью zip функции, давайте их соберем в кортежи по парам.

In [72]:
for pair in zip(type_ticket,price): #
    print(pair)

('Single ticket', 'HUF 350')
('Single ticket bought on the spot', 'HUF 450')
('Block of 10 tickets', 'HUF 3 000')
('Airport shuttle bus single ticket ', 'HUF 900')
('Transfer ticket', 'HUF 530')
('Short section metro ticket for up to 3 stops', 'HUF 300')
('Single ticket for public transport boat', 'HUF 750')
('Single ticket for public transport boat Children (under 15)', 'HUF 550')
('Metropolitan area ticket', 'HUF 250')
('Metropolitan area single ticket (50 % price discount)', 'HUF 125')
('Metropolitan area single ticket (90 % price discount)', 'HUF 25')
('Airport bus extension ticket', 'HUF 300')
('Pupil group travelcard', 'HUF 650/person')
('Budapest 24-hour travelcard', 'HUF 1 650')
('Budapest 24-hour group travelcard (for 1-5 passengers traveling together)', 'HUF 3 300')
('5/30 BKK 24-hour travelcard', 'HUF 4 550')
('Budapest 72-hour travelcard', 'HUF 4,150')
('Budapest 7-day travelcard', 'HUF 4 950')
('Budapest Card for 24 hours', 'HUF 6,490')
('Budapest Card for 48 hours', 'HUF 

**Отлично!** Теперь все билеты в одном месте, давайте сделаем упражнения.

## Exercise &#x1F4D8;
* Преобразуйте полученные данные в pandas dataframe.
* Выделите только годовые и квартальные билеты
* Сколько мы сэкономим, если купим годовой билет вместо четырех квартальных?

## Exercise &#x1F4D7;
* Если посмотреть на страницу, то мы увидим, что мы пропустили некторые билеты (например, Suburban railway estension). Исправьте код, чтобы и их добавить.

In [ ]:
import pandas as pd
df = pd.DataFrame(zip(type_ticket,price), columns = ['Ticket', 'Price'])

In [75]:
df[df['Ticket'].str.contains("Annual")]

,Ticket,Price
52,Annual all-line Budapest-pass,HUF 217 960
53,Annual all-line Budapest-pass (business),HUF 219 860
55,Annual Budapest-pass,HUF 9 500/month
56,Annual Budapest-pass (business),HUF 10 500/month


# PDF

## Download pdf

Импортируем модуль requests и откроем файл для записи, в который потом сохраним нашу pdf

In [ ]:
import requests

o = open("press.pdf",'wb')

Мы будем сохранять отчет Евро Парламента. У нас уже есть ссылка на саму pdf, поэтому просто укажем ее. С помощью модуля requests откроем ссылку для потокового скачивания файла (размер pdf может быть большим и не факт что он уместиться за раз в памяти)

In [ ]:
url=("http://www.europarl.europa.eu/pdfs/news/public/story/20161026STO49002/20161026STO49002_en.pdf" )
release=requests.get(url, stream=True)
press_release=release.content

Осталось все записать файл и закрыть его запись

In [ ]:

o.write(press_release)
o.close()

## Download all PDFs from page

Попробуем задание посложнее. Скачаем **все** pdf, которые есть на странице с курсами ВШЭ для направления политология. Зайдя на сайт и выбрав необходимые фильтры, получаем следующую страницы, на которой есть инормация о курсах:

https://www.hse.ru/edu/courses/index.html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0

In [ ]:
import requests
import bs4

url = ("https://www.hse.ru/edu/courses/index.html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0")
page = requests.get(url)
courses = page.text

Используем модуль bs4, а конкретно метод BeautifulSoup, который представит нам сайт в удобном для работы виде (информация будет разбита по тэгам веб страницы div, a и т.д.)

В списке links будет хранить ссылки на курсы, пройдя по которым мы сможем скачать pdf с программой курса

In [ ]:
links = [] 

soup = bs4.BeautifulSoup(courses, "html.parser")

Изучив код страницы, мы видим что ссылки на курсы лежат в теге "а". Поэтому найдем все такие теги и заберем адрес ссылки, которая хранится внутри тега "а" с параметром href. Проверим, ссылка ли это на курс (в этом теге могут быть ссылки на элементы меню) и есть ли такая ссылка в нашем списке, и если нет, то добавим ее.

**Всегда** придется изучать строение самой таблицы, чтобы обращаться и брать ее элементы.



In [ ]:
for name in soup.findAll ("a"):
    course = name.get ("href")
    if "hse.ru/edu/courses/" in str(course):
        if str(course) not in links:
            links.append(course)

Получили список ссылок на курсы со **всей** странице.

Теперь мы пройдемся по всем этим ссылкам, и точно также через request откроем их и прогоним через BeatifulSoup.
Открыв страницы с курсом видим, что в коде страницы опять же ссылка на **pdf** хранится в теге "a". Но есть таких ссылок две. Да, все так. В коде страницы есть два тега "а" в каждом из которых есть по ссылке на файт. На помощью нам придет возможность при поиске элемента через findAll указать класс тега. Поэтому в итоге каждый раз получим только 1 ссылку.

Ее мы как при скачивании pdf с сайта Евро Парламента просто откроем новым requests'ом и запишем в файл. Название файла будет точно таким же как он назывался на сайт

In [ ]:
for link in links: 
    ind_release = requests.get(str(link))  #проходим по ссылке
    ind_release = ind_release.text

    soup_release = bs4.BeautifulSoup(ind_release, "html.parser") # делаем все красивым
    for name in soup_release.findAll ('a', class_ = "link fileRef"): # находим все теги с нужным классом
        pdf_link = name.get ('href') # забираем ссылку
        if "data/" in  str(pdf_link): # проверяем что это нужная ссылка на файл (все ссылки на файл, начинаются с data/имя_файла)
            final_url = ("https://www.hse.ru/" + str(pdf_link)) # составляем финальную ссылку и делаем к ней request
            release = requests.get(final_url, stream=True)
            press_release = release.content
            o = open("" + final_url[final_url.rfind("/") + 1:], "wb") # final_url[final_url.rfind("/") + 1:] этим мы берем название файла из конца ссылки
            o.write(press_release)
o.close()

## Download all PDFs from several pages

Еще усложним задачу. Теперь мы будем собирать pdf с нескольких страницы выдачи. Код будет похож на прошлый пример и мы только обернем его циклом по страницам.

Заведем переменную *page* для хранения текущей страницы и будем увеличивать ее после завершения прохода текущей страницы.

Для выбора новой страницы просто подняем цифру в адресе сайт на нужную. Пример:
https://www.hse.ru/edu/courses/page1
https://www.hse.ru/edu/courses/page2 
https://www.hse.ru/edu/courses/page3 

И т.д.


In [ ]:
import requests
import bs4

page = 1
while page <= 3: #количество страниц, по которым нужно пройти
    
    #делаем запрос к нужной странице
    url = ("https://www.hse.ru/edu/courses/page" + str(page) + ".html?words=&full_words=&edu_year=2018&lecturer=&edu_level=78397&language=&level=1191462%3A130671177&mandatory=&is_dpo=0&filial=22723&modules=10162246&modules=10162247&xlc=&genelective=0")
    pages = requests.get(url)
    courses = pages.text
    
    #дальше код полностью с прошлого примера

    links = []  

    soup = bs4.BeautifulSoup(courses, "html.parser")

    for name in soup.findAll ("a"):
        course = name.get ("href")
        if "hse.ru/edu/courses/" in str(course):
            if str(course) not in links:
                links.append(course)
                
    for link in links:
        ind_release = requests.get(str(link))
        ind_release = ind_release.text

        soup_release = bs4.BeautifulSoup(ind_release, "html.parser")
        for name in soup_release.findAll ('a', class_ = "link fileRef"):
            pdf_link = name.get ('href')
            if "data/" in  str(pdf_link):
                final_url = ("https://www.hse.ru/" + str(pdf_link))
                release = requests.get(final_url, stream=True)
                press_release = release.content
                o = open("" + final_url[final_url.rfind("/") + 1:], "wb")
                o.write(press_release)
            
    page += 1 # "берем" новую страницу
    
o.close()
    